## Line Following

<img align="right" src="img/line_following.png" />
Up to now, we have worked up a line detection algorithm. Now that this scheme is up an running, we can move on to the task of driving the robot such that the line stays near the center of the camera image.

We propose to use [a *proportional* controller](https://en.wikipedia.org/wiki/Proportional_control), which means that a linear scaling of an error drives the control output. In this case, the error signal is the distance between the center of the image and the center of the line that we are trying to follow. The control output is the steering (angular velocity) of the robot.


In [1]:
import packages.initialization
import pioneer3dx as p3dx
p3dx.init()

In [2]:
import cv2
import numpy

### Image processing
Fill in the necessary code in the following function, which computes the centroid of the line of the image passed as an argument, as explained in the previous notebook.

In [3]:
def line_centroid(image):
    hsv = cv2.cvtColor(p3dx.image,cv2.COLOR_RGB2HSV) # Convert RGB to HSV
    lower_cyan = numpy.array([80,100,100]) # set threshold for cyan
    upper_cyan = numpy.array([100,255,255])
    mask = cv2.inRange(hsv,lower_cyan,upper_cyan) # create mask
    mask[0:80,0:150] = 0 # consider only nearest line segment
    M = cv2.moments(mask)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    return cx, cy

We need the code for the motion of the robot with the given linear and angular velocities, as in previous modules.

In [4]:
def move(V_robot,w_robot):
    r = 0.1953 / 2
    L = 0.33
    w_r = (2 * V_robot + L * w_robot) / (2*r)
    w_l = (2 * V_robot - L * w_robot) / (2*r)
    p3dx.move(w_l, w_r)

### Main loop
This is the main control loop. The error should be computed as:

$$ err = C_x - \frac{width}{2} $$

where $C_x$ is the $x$-coordinate of the centroid, and $width$ is the width of the image.

The linear velocity is constant, e.g. $2 m/s$ and the angular velocity $\omega$ is computed as:

$$ \omega = -K_p err $$

where $K_p$ is the gain of the proportional controller, which can be set to $0.01$.

In [6]:
p3dx.tilt(-0.47) # tilt down the Kinect
kp = 0.01
try:
    width = 150
    while True:
        cx, cy = line_centroid(p3dx.image)
        err = cx - (width/2)
        linear = 1
        angular = -kp*err
        move(linear, angular)
except KeyboardInterrupt:
    move(0,0)

ZeroDivisionError: float division by zero

Next: [Line Following with Obstacle Avoidance](Line%20Following%20Obstacle.ipynb)

---
#### Try-a-Bot: an open source guide for robot programming
Developed by:
[![Robotic Intelligence Lab @ UJI](img/logo/robinlab.png "Robotic Intelligence Lab @ UJI")](http://robinlab.uji.es)

Sponsored by:
<table>
<tr>
<td style="border:1px solid #ffffff ;">
<a href="http://www.ieee-ras.org"><img src="img/logo/ras.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="http://www.cyberbotics.com"><img src="img/logo/cyberbotics.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="http://www.theconstructsim.com"><img src="img/logo/theconstruct.png"></a>
</td>
</tr>
</table>

Follow us:
<table>
<tr>
<td style="border:1px solid #ffffff ;">
<a href="https://www.facebook.com/RobotProgrammingNetwork"><img src="img/logo/facebook.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="https://www.youtube.com/user/robotprogrammingnet"><img src="img/logo/youtube.png"></a>
</td>
</tr>
</table>